In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content

/content


In [ ]:
import os
import shutil
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
!pip install openslide-python
!sudo apt-get install openslide-tools
!sudo apt-get install python-openslide

In [ ]:
!git clone https://github.com/binli123/dsmil-wsi.git
%cd dsmil-wsi

Cloning into 'dsmil-wsi'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 595 (delta 12), reused 15 (delta 6), pack-reused 554
Receiving objects: 100% (595/595), 48.62 MiB | 15.75 MiB/s, done.
Resolving deltas: 100% (324/324), done.
/content/dsmil-wsi


In [ ]:
# 마운트 전용

os.makedirs('./WSI/TRAIN_DATA', exist_ok=True)
!unzip /content/drive/MyDrive/Data/DACON/train_imgs01.zip -d ./WSI/TRAIN_DATA
!unzip /content/drive/MyDrive/Data/DACON/train_imgs02.zip -d ./WSI/TRAIN_DATA
!unzip /content/drive/MyDrive/Data/DACON/DACON_csv.zip

In [ ]:
train_df = pd.read_csv('./train.csv')[['ID', 'N_category']]
train_df[:]['N_category'].value_counts()

In [ ]:
for i, j in tqdm(train_df.values[:]):
  os.makedirs(f'./WSI/TRAIN_DATA/{j}', exist_ok=True)
  shutil.move(f'./WSI/TRAIN_DATA/{i}.png', f'./WSI/TRAIN_DATA/{j}')

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
!python deepzoom_tiler.py -m 0 -b 20 -v png -d TRAIN_DATA # -q 100 --f png

In [ ]:
%cd simclr
!sed -i 's/batch_size: 4096/batch_size: 256/' config.yaml
!sed -i 's/epochs: 100/epochs: 15/' config.yaml
!sed -i 's/gpu_ids: (0)/gpu_ids: (0,)/' config.yaml

/content/dsmil-wsi/simclr


In [ ]:
!git clone https://github.com/NVIDIA/apex.git

Cloning into 'apex'...
remote: Enumerating objects: 10636, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 10636 (delta 89), reused 80 (delta 37), pack-reused 10478
Receiving objects: 100% (10636/10636), 15.16 MiB | 16.96 MiB/s, done.
Resolving deltas: 100% (7316/7316), done.


In [ ]:
# !sed -i 's/pretrained=False/weights=None/g' ./models/resnet_simclr.py

In [ ]:
!python run.py --dataset=TRAIN_DATA

In [ ]:
%cd ..
!python compute_feats.py --dataset=TRAIN_DATA --num_classes 1

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Use pretrained features.
 Computed: 20/1000 -- 1/1No valid patch extracted from: WSI/TRAIN_DATA/single/1/BC_01_0837
 Computed: 1000/1000 -- 1/1

## Train

In [ ]:
# !sed -i "s/init.pth/os.path.join('embedder','TRAIN_DATA','embedder.pth')/" train_tcga.py

In [ ]:
!python train_tcga.py --dataset=TRAIN_DATA --num_epochs 10 --num_classes 1\
# --lr 0.0002

train_tcga.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  bag_label = Variable(Tensor([label]))
 Epoch [1/10] train loss: 0.7032 test loss: 0.6941, average score: 0.5850, AUC: class-0>>0.58984375
Best model saved at: weights/11282022/1.pth
Best thresholds ===>>> class-0>>0.5153811573982239
 Epoch [2/10] train loss: 0.7025 test loss: 0.7371, average score: 0.6600, AUC: class-0>>0.6796875
Best model saved at: weights/11282022/1.pth
Best thresholds ===>>> class-0>>0.6325733661651611
 Epoch [3/10] train loss: 0.7000 test loss: 0.7113, average score: 0.6800, AUC: class-0>>0.69140625
Best model saved at: weights/11282022/1.pth
Best thresholds ===>>> class-0>>0.5871957540512085
 Epoch [4/10] train loss: 0.6968 test loss: 0.7387, average score: 0.6600, AUC: class-0>>0.68800080

In [ ]:
os.makedirs('./WSI/TEST_DATA/image', exist_ok=True)
os.makedirs('./test/input', exist_ok=True)
!unzip /content/drive/MyDrive/Data/DACON/test_imgs.zip -d ./WSI/TEST_DATA/image

In [ ]:
!python deepzoom_tiler.py -m 0 -b 20 -v png -d TEST_DATA -f jpg
# -s 64

Process slide 1/250
Tiling slide: wrote 200/302 tiles
 Organizing patches
 Patch [56/56]Done.
Process slide 2/250
Tiling slide: wrote 200/365 tiles
 Organizing patches
 Patch [94/94]Done.
Process slide 3/250
Tiling slide: wrote 300/476 tiles
 Organizing patches
 Patch [137/137]Done.
Process slide 4/250
Tiling slide: wrote 400/597 tiles
 Organizing patches
 Patch [145/145]Done.
Process slide 5/250
Tiling slide: wrote 300/511 tiles
 Organizing patches
 Patch [159/159]Done.
Process slide 6/250
Tiling slide: wrote 300/448 tiles
 Organizing patches
 Patch [103/103]Done.
Process slide 7/250
Tiling slide: wrote 200/379 tiles
 Organizing patches
 Patch [81/81]Done.
Process slide 8/250
Tiling slide: wrote 300/526 tiles
 Organizing patches
 Patch [81/81]Done.
Process slide 9/250
Tiling slide: wrote 200/353 tiles
 Organizing patches
 Patch [67/67]Done.
Process slide 10/250
Tiling slide: wrote 100/280 tiles
 Organizing patches
 Patch [48/48]Done.
Process slide 11/250
Tiling slide: wrote 300/430 ti

In [ ]:
!python attention_map.py\
--bag_path WSI/TEST_DATA/single/image\
--map_path test/output\
--embedder_weights embedder/TRAIN_DATA/embedder.pth\
--aggregator_weights weights/*/*.pth\
--export_scores 1\
--class_name N P\
--num_classes 1\
--thres 0.5\